In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import re
import string

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from keras.layers import Conv1D, GlobalMaxPool1D
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Carregando os Dados

In [ ]:
treino = pd.read_csv('../input/nlp-getting-started/train.csv')

In [ ]:
treino

# Deletando Colunas

In [ ]:
treino.drop(columns=['id','keyword','location'], inplace=True)

In [ ]:
treino

# Visualizando os Dados

In [ ]:
print(treino.target.value_counts())
treino.target.value_counts().plot(kind='bar')

In [ ]:
treino.isnull().sum()

# Limpando os dados

In [ ]:
abreviacao = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired",
    "ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how does",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so is",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    " u ": " you ",
    " ur ": " your ",
    " n ": " and ",
    "won't": "would not",
    'dis': 'this',
    'bak': 'back',
    'brng': 'bring'}

In [ ]:
treino.text = treino.text.apply(lambda x: x.lower())
treino.text = treino.text.replace(regex=abreviacao) 
treino.text = treino.text.apply(lambda x: re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', x))
treino.text = treino.text.apply(lambda x: re.sub('(@[A-Za-z]+[A-za-z0-9-_]+)', '', x))
treino.text = treino.text.apply(lambda x: re.sub('rt','',x))
treino.text = treino.text.apply(lambda x: re.sub('\<http.+?\>', '', x))
treino.text = treino.text.apply(lambda x: ''.join([i for i in x if i not in string.punctuation]))
treino.text = treino.text.apply(lambda x: re.sub(' +', ' ', x))

In [ ]:
tm = []
for t in treino.text:
    tm.append(len(t.split(' ')))
tamanho = np.max(tm)
print('Tamanho máximo da sentença:', tamanho)

# Tokens

In [ ]:
# Número máximo de palavras usadas
max_palavra = 15000

# Tamanho da dimensão
embedding_dim = 20

tk = Tokenizer(num_words=max_palavra)
tk.fit_on_texts(treino.text.values)
pl_unicas = tk.word_index
print('Quantidade de palavras únicas: ', len(pl_unicas)+1)

In [ ]:
# Contando palavras
minimo = 2

contagem = 0 
total_contagem = 0
frequencia = 0
total_frequencia = 0

for key, value in tk.word_counts.items():
    total_contagem += 1
    total_frequencia = total_frequencia + value
    if (value < minimo):
        contagem += 1
        frequencia = frequencia + value
        
print("% de palavras raras:",(contagem/total_contagem)*100)
print("Total de palavras raras:",(frequencia/total_frequencia)*100)
print('Palavras raras: ',contagem)
print('Tamanho do vocabulário: ',total_contagem)
print('Palavras mais comuns: ', total_contagem-contagem)

In [ ]:
tk = Tokenizer(total_contagem-contagem)
tk.fit_on_texts(list(treino.text))
pl_unicas = tk.word_index
print('Quantidade de palavras únicas: ', len(pl_unicas)+1)

# Padding

In [ ]:
x = tk.texts_to_sequences(treino.text.values)
x = pad_sequences(x, maxlen=tamanho, padding='post')
print('Quantidade de dados de x: ', x.shape)

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
y = lb.fit_transform(treino.target.values)

#y = pd.get_dummies(treino.target.values)
#print('Quantidade de dados de y: ', y.shape)

In [ ]:
y

In [ ]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.2, stratify=y)
print(x_treino.shape, y_treino.shape)
print(x_teste.shape, y_teste.shape)

# N-Gram

In [ ]:
batch_size = 128

parada = EarlyStopping(monitor='val_accuracy', mode='min', patience=10, baseline=0.4, min_delta=.0001)
redutor = ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=1, factor=0.5, min_lr=0.0001)

In [ ]:
modelo_ngram = Sequential()
modelo_ngram.add(Embedding(10000,30))
modelo_ngram.add(Conv1D(filters=15, kernel_size=2, padding='same', activation='relu'))
modelo_ngram.add(Conv1D(filters=15, kernel_size=3, padding='same', activation='relu'))
modelo_ngram.add(Conv1D(filters=15, kernel_size=4, padding='same', activation='relu'))
modelo_ngram.add(GlobalMaxPool1D())
modelo_ngram.add(Dense(128, activation='relu'))
modelo_ngram.add(Dropout(0.1))
modelo_ngram.add(Dense(1, activation='sigmoid'))
modelo_ngram.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

modelo_n = modelo_ngram.fit(x_treino, y_treino, epochs=100, batch_size=batch_size, verbose=1,
                  validation_data=(x_teste, y_teste),callbacks=[redutor, parada])

loss, acc = modelo_ngram.evaluate(x_teste, y_teste)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15,5))
axes[0].plot(modelo_n.history['accuracy'])
axes[0].plot(modelo_n.history['val_accuracy'])
axes[0].set_xlabel('Epochs')
axes[0].set_ylabel('Acurácia')
axes[0].legend(['Acurácia em Treino','Acurácia em Teste'])
axes[0].grid(True)

axes[1].plot(modelo_n.history['loss'])
axes[1].plot(modelo_n.history['val_loss'])
axes[1].set_xlabel('Epochs')
axes[1].set_ylabel('Erro')
axes[1].legend(['Erro em Treino','Erro em Teste'])
axes[1].grid(True)

In [ ]:
teste = pd.read_csv('../input/nlp-getting-started/test.csv')

In [ ]:
teste

In [ ]:
teste.drop(columns=['id','keyword','location'], inplace=True)

teste.text = teste.text.apply(lambda x: x.lower())
teste.text = teste.text.replace(regex=abreviacao) 
teste.text = teste.text.apply(lambda x: re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', x))
teste.text = teste.text.apply(lambda x: re.sub('(@[A-Za-z]+[A-za-z0-9-_]+)', '', x))
teste.text = teste.text.apply(lambda x: re.sub('rt','',x))
teste.text = teste.text.apply(lambda x: re.sub('\<http.+?\>', '', x))
teste.text = teste.text.apply(lambda x: ''.join([i for i in x if i not in string.punctuation]))
teste.text = teste.text.apply(lambda x: re.sub(' +', ' ', x))

In [ ]:
tm = []
for t in teste.text:
    tm.append(len(t.split(' ')))
tamanho = np.max(tm)
print('Tamanho máximo da sentença:', tamanho)

In [ ]:
#t = Tokenizer(total_contagem-contagem)
#t.fit_on_texts(list(treino.text))
#pl_unicas = t.word_index
#print('Quantidade de palavras únicas: ', len(pl_unicas)+1)

pre = tk.texts_to_sequences(teste.text.values)
pre = pad_sequences(pre, maxlen=38, padding='post')

In [ ]:
previsto = modelo_ngram.predict(pre)

In [ ]:
previsto = (np.round(previsto)).astype(int)
previsto

In [ ]:
submission = pd.read_csv('../input/nlp-getting-started/sample_submission.csv')

In [ ]:
submission['target'] = previsto

In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv', index=False)